**Motivation**: Due to the slowness of direct implementaion of BFO, I feel an urgent need for vectorized version of BFO. 

### Define a convenient function for randomly generating points with pre-assigned Range

In [1]:
## m: number of random points to generate
## Range: dim x 2 matrix; the d-th row is the range of the d-th variable
function RandPts(Range::Array{Float64,2}, m::Int)
    dim = size(Range,1)
    randPts = rand(dim,m)
    for d = 1:dim
        Original = randPts[d,:]
        randPts[d,:] = (Range[d,2]-Range[d,1])*Original.+Range[d,1]
    end
    return randPts
end

RandPts (generic function with 1 method)

In [6]:
Range = Array{Float64,2}([0 3; 1 4; 2 5])
m = 10;

In [11]:
@time RandPts(Range,m)

  0.000009 seconds (14 allocations: 1.891 KiB)


3×10 Array{Float64,2}:
 2.9142   0.843835  1.89914  1.31308  …  0.0607998  1.15026  0.928093
 2.37882  1.2721    2.69931  1.15669     1.96483    1.97681  3.81663 
 4.09613  2.88865   2.25775  2.30644     4.23734    4.92265  3.07238 

In [2]:
"""
**RandUnit** is a function generating m random unit vectors in dimension d. <br>
**Inputs**: <br>
1. d: dimension <br>
2. m: number of vectors to generate <br>
**Output**: <br>
an d x m matrix whose columns are the desired vectors. <br>
"""
function RandUnit(d,m)
    Rand = randn(d,m)
    for k = 1:m
        Rand[:,k]./=norm(Rand[:,k])
    end
    return Rand
end

RandUnit

In [44]:
?RandUnit

search: RandUnit



**RandUnit** is a function generating m random unit vectors in dimension d. <br> **Inputs**: <br>

1. d: dimension <br>
2. m: number of vectors to generate <br>

**Output**: <br> an d x m matrix whose columns are the desired vectors. <br>


Notations recap:
1. **J**: a function with n variables
2. **Range**: exploration range <br><br>

3. **n** = 2, dimension of the input of J
4. **S** = 10, number of bacteria
5. **Sr** = 4,  number of bacteria removed in reproductive step
6. **Nc** = 20, number of chemotactic steps
7. **Ns** = 5, number of swim steps
8. **Nre** = 5, number of reproductive steps
9. **Ned** = 2, elimination and dispersal steps
10. **Ped** = 0.3, probability of elimination
11. **Ci** = (Range[2]-Range[1])/S, the run-length unit

In [79]:
function BFO(J, Range, n = 2::Int, S = 10::Int, Sr = 4::Int, Nc = 20::Int, Ns = 5::Int, 
        Nre = 5::Int, Ned = 2::Int, Ped = 0.3::Float64, Ci = ((Range[2]-Range[1])/S)::Float64)
    
    ## randomly generate S bacteria in Range
    B_loc = RandPts(Range,S)
    ## a dictionary recording the path of bacterium i
    Path_Dict = Dict(i=>[zeros(n,0) B_loc[:,i]] for i=1:S)
    for l = 1:Ned ## index of elimination-dispersal steps
        for k = 1:Nre ## index of reproductive steps
            for j = 1:Nc ## index of chemotactic steps
                ## Chemotactic Step
                println("Chemo-step $j")
                ToSwim = collect(1:S) ## monitor the swimming bacteria
                Tumble = RandUnit(n,S)
                m = 0 ## index of swimming
                while (ToSwim!=Array{Int64,1}())&(m<Ns) ## tumble/swim
                    m = m + 1
                    J_old = [J(B_loc[:,i]) for i in ToSwim]
    
                    B_loc_new = copy(B_loc)
                    B_loc_new[:,ToSwim] = B_loc[:,ToSwim].+Ci*Tumble[:,ToSwim]
                    ## Mirror back the out-of-range bacteria
                    for d = 1:n
                        ## mirror back bacteria tumbling out from below
                        B_loc_new[d,ToSwim] = (abs.(B_loc_new[d,ToSwim].-Range[d,1])).+(Range[d,1]) 
                        ## mirror back bacteria tumbling out from above
                        B_loc_new[d,ToSwim] = Range[d,2].-(abs.(Range[d,2].-B_loc_new[d,ToSwim]))
                    end
                    B_loc_new
    
                    ## Evaluate J at new locations
                    J_new = [J(B_loc_new[:,i]) for i in ToSwim]
    
                    ## Find out improved bacteria and update the swimming ones
                    ToSwim = ToSwim[findall(J_new.<J_old)]
    
                    for i in ToSwim
                        Path_Dict[i] = [Path_Dict[i] B_loc_new[:,i]]
                    end
                    B_loc[:,ToSwim] = B_loc_new[:,ToSwim]
                end ## end of tumble/swim
            end ## end of chemotactic steps
            
            if k<Nre
                ## Reproductive Step
                println("Repro-step $k")
                ## (!!!) I define the health of a bacterium as the J value of its current location
                Health = [J(B_loc[:,i]) for i=1:S]
                Health_sort = sortslices([Health collect(1:S)], dims = 1) ## sort the bacteria according to health
                B_survive = Array{Int,1}(Health_sort[1:S-Sr,2]) ## ## pick out the most healthy (S-Sr) bacteria
                B_die = setdiff(collect(1:S), B_survive)
                B_rep = sort([B_survive;B_survive[1:Sr]]) ## reproduce the most healthy Sr bacteria
                B_loc[:,B_die] = B_loc[:,B_survive[1:Sr]]
                
                ## update the path dictionary
                for i in B_die
                    Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                end
            end
        end
        ## Elimination-Dispersal Step
        println("ED-step $l")
        if l<Ned
            Alive = [sample([false, true], aweights([Ped,1-Ped])) for i=1:S] # true = alive, false = kill
            Killed = setdiff(collect(1:S),Alive) ## kiilled bacteria
            ## randomly generate a bacterium for each killed bacterium
            Alive = findall(Alive)
            
            ## Rebuild the killed bacteria in random locations in Range
            B_loc[:,Killed] = RandPts(Range,length(Killed))
            ## update the path dictionary
            for i in Killed
                Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
            end
        end
    end
    B_best = Int(sortslices([[J(B_loc[:,i]) for i=1:S] collect(1:S)], dims = 1)[1,2]) ## best bacterium
    X_best = B_loc[:,B_best] ## best location for minimizing J
    J_best = J(X_best) ## best (minimum) J value
    return Dict("Minimum"=>J_best, "Minimum Point"=>X_best, "Path_Dict"=>Path_Dict)
end

BFO (generic function with 10 methods)

In [75]:
using LinearAlgebra # "norm"
using StatsBase
Range = Array{Float64,2}([0 2;1 3])
n = 2#::Int
S = 10#::Int, 
Sr = 4#::Int, 
Nc = 20#::Int, 
Ns = 5#::Int
Nre = 2#::Int
Ned = 2#::Int
Ped = 0.3#::Float64
Ci = ((norm(Range[:,2]-Range[:,1]))/(2*S));#::Float64 

In [73]:
J(X) = sum(X)

J (generic function with 1 method)

In [80]:
@time BFO(J,Range)

Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 1
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 2
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 3
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Che

Dict{String,Any} with 3 entries:
  "Path_Dict"     => Dict(7=>[0.759961 0.682776 … 0.000460271 0.00117275; 1.866…
  "Minimum Point" => [0.00117275, 1.00434]
  "Minimum"       => 1.00552

In [58]:
for i=1:S
    Path_i= Path_Dict[i]
    println([J(Path_i[:,k]) for k=1:size(Path_i,2)])
end

[2.59303, 2.39607]
[2.72201, 2.67022, 2.61843]
[3.60451]
[2.75717, 2.66596, 2.57475, 2.48354, 2.39234, 2.32731]
[2.52964, 2.43219, 2.33474, 2.23729, 2.13984, 2.04239]
[3.52072]
[1.3147]
[2.05796]
[3.03446, 2.88936, 2.74425, 2.59914, 2.45404, 2.30893]
[1.79963, 1.66943, 1.53923, 1.40903, 1.27883, 1.2166]


In [59]:
using Plots
plotly()
scatter(B_loc[1,:], B_loc[2,:], xlim = Range[1,:], ylim = Range[2,:])
for i = 1:S
    Path_i = Path_Dict[i]
    if size(Path_i,2)>1
        plot!(Path_i[1,:], Path_i[2,:])
        ## scatter!(Path_i[1,:], Path_i[2,:])
    end
end
plot!()